# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import weakref
import contextlib

In [ ]:
class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config('enable_backprop', False)

In [ ]:
#| export
class Variable:
    def __init__(self, data, name=None):
        if data is not None: # Why allowing "None" here?
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
        if self.grad == None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f in seen_set:
                return
            funcs.append(f)
            seen_set.add(f)
            funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()

            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx if x.grad==None else x.grad+gx
                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

    def cleargrad(self):
        self.grad = None

In [ ]:
#| export
class Function:
    def __call__(self, *inputs):
        def as_array(y): return np.array(y) if np.isscalar(y) else y # for numpy spec
        def as_variable(obj): return obj if isinstance(obj, Variable) else Variable(obj)

        inputs = [as_variable(x) for x in inputs]
        xs = [input.data for input in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([o.generation for o in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, in_data):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [ ]:
#| export
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

def add(x0, x1):
    return Add()(x0, x1)

In [ ]:
#| export
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1 * gy, x0 * gy

def mul(x0, x1):
    return Mul()(x0, x1)

In [ ]:
#| export
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)

In [ ]:
#| export
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)

In [ ]:
Variable.__add__ = add
Variable.__mul__ = mul

In [ ]:
# Test Variable
x = Variable(np.array(1.0))
assert x.data == np.array(1.0)

x = Variable(None)
assert x.data == None

try:
    Variable(1.0)
except Exception as e:
    assert f"{e}"=="<class 'float'> is not supported"

In [ ]:
x0 = Variable(np.array(2)) 
x1 = Variable(np.array(3))
y = add(x0, x1)
assert y.data==5

In [ ]:
# Test Square
x = Variable(np.array(10))
y = square(x)
assert type(y)==Variable
assert y.data==100

In [ ]:
# Test Exp
x = Variable(np.array(2.))
y = exp(x)
assert np.allclose(y.data, 7.3890561)

In [ ]:
# Test concatenate
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)
assert np.allclose(y.data, 1.648721270700128)

In [ ]:
#| export
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(np.array(x.data - eps))
    x1 = Variable(np.array(x.data + eps))
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [ ]:
# test numerical_diff
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)

assert np.allclose(dy, 4)

In [ ]:
# concatenate test with numerical_diff
def f(x): return square(exp(square(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
assert np.allclose(dy, 3.2974426293330694)

In [ ]:
# Test backward
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.backward()
assert np.allclose(x.grad, 3.297442541400256)

In [ ]:
# step14: Test use the same variable repeatedly
x = Variable(np.array(3.0))
y = add(x, x)
assert y.data==6.0
y.backward()
assert x.grad==2.0, print(x.grad)

x.cleargrad()
y = add(add(x, x), x)
assert y.data==9.0
y.backward()
assert x.grad==3.0, print(x.grad)

In [ ]:
# step16: Test multiple backward()s in order
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()
#print(y.data)
assert x.grad==64.0, print(x.grad)

In [ ]:
# step17: weekref test
a = np.array([1, 2, 3])
b = weakref.ref(a)
assert "numpy.ndarray" in f"{b}"
assert (b()==np.array([1,2,3])).all() # kinda pointer?
a = None
assert 'dead' in f"{b}"

In [ ]:
# step18: unnecessary intermediate grads
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()
assert y.grad==None and t.grad==None
assert x0.grad==2.0 and x1.grad==1.0

In [ ]:
# step18.4: retain_grad test
with no_grad():
    x = Variable(np.ones((100, 100, 100)))
    y = square(square(square(x)))

In [ ]:
@contextlib.contextmanager
def config_test():
    print('start')
    try:
        print('going back to the caller')
        yield
    finally:
        print('done')
        
with config_test():
    print('process...')

start
going back to the caller
process...
done


In [ ]:
# step19: test Variable shape
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape)
print(x)

(2, 3)
variable([[1 2 3]
          [4 5 6]])


In [ ]:
# step20: test multiplication
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))
y = a * b + c
y.backward()
assert y.data==7.0 and a.grad==2.0 and b.grad==3.0 and c.grad==1.0

In [ ]:
# step21: test casting to Variable
x = Variable(np.array(2.0))
y = x + 3.0
assert y.data==5.

TypeError: <class 'float'> is not supported

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()